# Crawling State of the Union Adresses from millercenter.org website

In [2]:
# Loading packages

import requests
from bs4 import BeautifulSoup
import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import re
import pandas as pd
import numpy as np

In [3]:
### Define our Connector

def ratelimit(dt):
    "A function that handles the rate of your calls."
    time.sleep(dt) # sleep one second.

class Connector():
  def __init__(self,logfile,overwrite_log=False,connector_type='requests',session=False,path2selenium='',n_tries = 5,timeout=30,waiting_time=0.5):
    """This Class implements a method for reliable connection to the internet and monitoring. 
    It handles simple errors due to connection problems, and logs a range of information for basic quality assessments
    
    Keyword arguments:
    logfile -- path to the logfile
    overwrite_log -- bool, defining if logfile should be cleared (rarely the case). 
    connector_type -- use the 'requests' module or the 'selenium'. Will have different since the selenium webdriver does not have a similar response object when using the get method, and monitoring the behavior cannot be automated in the same way.
    session -- requests.session object. For defining custom headers and proxies.
    path2selenium -- str, sets the path to the geckodriver needed when using selenium.
    n_tries -- int, defines the number of retries the *get* method will try to avoid random connection errors.
    timeout -- int, seconds the get request will wait for the server to respond, again to avoid connection errors.
    """
    
    ## Initialization function defining parameters. 
    self.n_tries = n_tries # For avoiding triviel error e.g. connection errors, this defines how many times it will retry.
    self.timeout = timeout # Defining the maximum time to wait for a server to response.
    self.waiting_time = waiting_time # define simple rate_limit parameter.
    ## not implemented here, if you use selenium.
    if connector_type=='selenium':
      assert path2selenium!='', "You need to specify the path to you geckodriver if you want to use Selenium"
      from selenium import webdriver 
      ## HIN download the latest geckodriver here: https://github.com/mozilla/geckodriver/releases

      assert os.path.isfile(path2selenium),'You need to insert a valid path2selenium the path to your geckodriver. You can download the latest geckodriver here: https://github.com/mozilla/geckodriver/releases'
      self.browser = webdriver.Firefox(executable_path=path2selenium) # start the browser with a path to the geckodriver.

    self.connector_type = connector_type # set the connector_type
    
    if session: # set the custom session
      self.session = session
    else:
      self.session = requests.session()
    self.logfilename = logfile # set the logfile path
    ## define header for the logfile
    header = ['id','project','connector_type','t', 'delta_t', 'url', 'redirect_url','response_size', 'response_code','success','error']
    if os.path.isfile(logfile):        
      if overwrite_log==True:
        self.log = open(logfile,'w')
        self.log.write(';'.join(header))
      else:
        self.log = open(logfile,'a')
    else:
      self.log = open(logfile,'w')
      self.log.write(';'.join(header))
    ## load log 
    with open(logfile,'r') as f: # open file
        
      l = f.read().split('\n') # read and split file by newlines.
      ## set id
      if len(l)<=1:
        self.id = 0
      else:
        self.id = int(l[-1][0])+1
            
  def get(self,url,project_name):
    """Method for connector reliably to the internet, with multiple tries and simple error handling, as well as default logging function.
    Input url and the project name for the log (i.e. is it part of mapping the domain, or is it the part of the final stage in the data collection).
    
    Keyword arguments:
    url -- str, url
    project_name -- str, Name used for analyzing the log. Use case could be the 'Mapping of domain','Meta_data_collection','main data collection'. 
    """
     
    project_name = project_name.replace(';','-') # make sure the default csv seperator is not in the project_name.
    if self.connector_type=='requests': # Determine connector method.
      for _ in range(self.n_tries): # for loop defining number of retries with the requests method.
        ratelimit(self.waiting_time)
        t = time.time()
        try: # error handling 
          response = self.session.get(url,timeout = self.timeout) # make get call

          err = '' # define python error variable as empty assumming success.
          success = True # define success variable
          redirect_url = response.url # log current url, after potential redirects 
          dt = t - time.time() # define delta-time waiting for the server and downloading content.
          size = len(response.text) # define variable for size of html content of the response.
          response_code = response.status_code # log status code.
          ## log...
          call_id = self.id # get current unique identifier for the call
          self.id+=1 # increment call id
          #['id','project_name','connector_type','t', 'delta_t', 'url', 'redirect_url','response_size', 'response_code','success','error']
          row = [call_id,project_name,self.connector_type,t,dt,url,redirect_url,size,response_code,success,err] # define row to be written in the log.
          self.log.write('\n'+';'.join(map(str,row))) # write log.
          self.log.flush()
          return response,call_id # return response and unique identifier.

        except Exception as e: # define error condition
          err = str(e) # python error
          response_code = '' # blank response code 
          success = False # call success = False
          size = 0 # content is empty.
          redirect_url = '' # redirect url empty 
          dt = t - time.time() # define delta t

          ## log...
          call_id = self.id # define unique identifier
          self.id+=1 # increment call_id

          row = [call_id,project_name,self.connector_type,t,dt,url,redirect_url,size,response_code,success,err] # define row
          self.log.write('\n'+';'.join(map(str,row))) # write row to log.
          self.log.flush()
    else:
      t = time.time()
      ratelimit(self.waiting_time)
      self.browser.get(url) # use selenium get method
      ## log
      call_id = self.id # define unique identifier for the call. 
      self.id+=1 # increment the call_id
      err = '' # blank error message
      success = '' # success blank
      redirect_url = self.browser.current_url # redirect url.
      dt = t - time.time() # get time for get method ... NOTE: not necessarily the complete load time.
      size = len(self.browser.page_source) # get size of content ... NOTE: not necessarily correct, since selenium works in the background, and could still be loading.
      response_code = '' # empty response code.
      row = [call_id,project_name,self.connector_type,t,dt,url,redirect_url,size,response_code,success,err] # define row 
      self.log.write('\n'+';'.join(map(str,row))) # write row to log file.
      self.log.flush()
    # Using selenium it will not return a response object, instead you should call the browser object of the connector.
    ## connector.browser.page_source will give you the html.
      return None,call_id

## Checking robots.txt

In [4]:
url = "https://millercenter.org/robots.txt"
r = requests.get(url)
robots = BeautifulSoup(r.text, 'html5lib')

print(robots)

<html><head></head><body>#
# robots.txt
#
# This file is to prevent the crawling and indexing of certain parts
# of your site by web crawlers and spiders run by sites like Yahoo!
# and Google. By telling these "robots" where not to go on your site,
# you save bandwidth and server resources.
#
# This file will be ignored unless it is at the root of your host:
# Used:    http://example.com/robots.txt
# Ignored: http://example.com/site/robots.txt
#
# For more information about the robots.txt standard, see:
# http://www.robotstxt.org/robotstxt.html

User-agent: *
# CSS, JS, Images
Allow: /core/*.css$
Allow: /core/*.css?
Allow: /core/*.js$
Allow: /core/*.js?
Allow: /core/*.gif
Allow: /core/*.jpg
Allow: /core/*.jpeg
Allow: /core/*.png
Allow: /core/*.svg
Allow: /profiles/*.css$
Allow: /profiles/*.css?
Allow: /profiles/*.js$
Allow: /profiles/*.js?
Allow: /profiles/*.gif
Allow: /profiles/*.jpg
Allow: /profiles/*.jpeg
Allow: /profiles/*.png
Allow: /profiles/*.svg
# Directories
Disallow: /core/
D

In [5]:
# It is not prohibited to crawl data from /the-presidency/presidential-speeches/
# Therefore we are fine to continue!

In [6]:
# creating directory for storing speeches

cwd =  os.getcwd()
if not os.path.exists(str(cwd) + "/speeches"):
    os.mkdir(str(cwd) + "/speeches")


In [7]:
# setting default download folder for chromedriver
# (do not forget to place the chromedriver in the working directory if not yet done)

chromeOptions = webdriver.ChromeOptions()
preferences = {"download.default_directory" : os.path.join(str(cwd),"/speeches")}
chromeOptions.add_experimental_option("prefs",preferences)

In [8]:
# connecting to browser

browser = webdriver.Chrome(executable_path = os.path.join(os.getcwd(), "chromedriver"), options=chromeOptions) 

In [9]:
# opening the https://millercenter.org/the-presidency/presidential-speeches website in chrome

browser.get("https://millercenter.org/the-presidency/presidential-speeches")

In [10]:
# This website is dynamic (as are the individual presidents sites). The further we scroll down 
# the more links to speeches are shown.

In [11]:
pause = 4 # had some interruptions with less than 3 seconds, with 3 and 4 s I made it to the bottom of the page

# Scrolling step by step automatically down, until the end of the page is reached
# Get scroll height
last_height = browser.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom of the page
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(pause)

    # Calculate new scroll height and compare with previous scroll height
    new_height = browser.execute_script("return document.body.scrollHeight")
    
    if new_height == last_height:
        # Getting the last version of the html code of the site after scrolling finished
        html = browser.page_source
        soup = BeautifulSoup(html, 'html5lib')
        links = [link.get('href') for link in soup.find_all('a')]
        break
        
    last_height = new_height

In [12]:
# Getting all links out of the soup and deleting None's
links = [link.get('href') for link in soup.find_all('a')]
links = list(filter(None, links)) 

In [13]:
# Filtering for 'state-union-address' as part if the link and adding to stateoftheunion list

stateoftheunion = []

for l in links:
    if 'state-union-address' in l or "inaugural-address" in l: 
        stateoftheunion.append("https://millercenter.org"+l)


In [14]:
stateoftheunion

['https://millercenter.org/the-presidency/presidential-speeches/february-5-2019-state-union-address',
 'https://millercenter.org/the-presidency/presidential-speeches/january-30-2018-state-union-address',
 'https://millercenter.org/the-presidency/presidential-speeches/january-20-2017-inaugural-address',
 'https://millercenter.org/the-presidency/presidential-speeches/january-12-2016-2016-state-union-address',
 'https://millercenter.org/the-presidency/presidential-speeches/january-20-2015-2015-state-union-address',
 'https://millercenter.org/the-presidency/presidential-speeches/january-28-2014-2014-state-union-address',
 'https://millercenter.org/the-presidency/presidential-speeches/february-13-2013-2013-state-union-address',
 'https://millercenter.org/the-presidency/presidential-speeches/january-21-2013-second-inaugural-address',
 'https://millercenter.org/the-presidency/presidential-speeches/january-24-2012-2012-state-union-address',
 'https://millercenter.org/the-presidency/presidentia

In [15]:
# Question: why are some years missing? e.g. 2001, 1993, 1981 ? First year of new presidency no sotu speech?

In [16]:
# Next step: crawling / looping over all links and getting the speech text, storing it with some meaningful name
# presidents name + date.txt?

In [17]:
data = []

connector = Connector('log_sotu.csv')

for link in stateoftheunion:
    r,call_id = connector.get(link,'sotu')
    #r = requests.get(link)
    soup = BeautifulSoup(r.content, "html5lib")

    name = soup.find('p',attrs={"class":"president-name"})
    name = name.get_text("president-name")
    
    date = soup.find('p',attrs={"class":"episode-date"})
    date = date.get_text("episode-date")
    
    # if clause since transcript website is different for Hoover
    
    diff = ["Herbert Hoover", "Calvin Coolidge", "Warren G. Harding", "Abraham Lincoln", "William McKinley",
           "Theodore Roosevelt", "William Taft", "Woodrow Wilson", "William McKinley", "Ulysses S. Grant",
           "Rutherford B. Hayes", "James A. Garfield", "Grover Cleveland", "Benjamin Harrison",
           "Zachary Taylor", "Franklin Pierce", "James K. Polk", "William Harrison", "Martin Van Buren", 
           "Andrew Jackson", "John Quincy Adams", "James Monroe", "James Madison", "Thomas Jefferson",
           "John Adams", "George Washington", "Calvin Coolidge", "James Buchanan", "Franklin Pierce"]
    
    if name in diff:
        speech = soup.find('div',attrs={"class":"view-transcript"})
        speech = speech.get_text("view-transcript")
    else: 
        speech = soup.find('div',attrs={"class":"transcript-inner"})
        speech = speech.get_text("transcript-inner")
    
    if 'state-union-address' in link: type = "State of the Union" 
    else: type = "Inaugural Address"

    data.append([name, date, speech, type])
    
    print(name+" "+ date)

df = pd.DataFrame(data, columns = ['Name', 'Date', "Speech", "Type"])

Donald Trump February 05, 2019
Donald Trump January 30, 2018
Donald Trump January 20, 2017
Barack Obama January 12, 2016
Barack Obama January 20, 2015
Barack Obama January 28, 2014
Barack Obama February 13, 2013
Barack Obama January 21, 2013
Barack Obama January 24, 2012
Barack Obama January 25, 2011
Barack Obama January 27, 2010
Barack Obama January 20, 2009
George W. Bush January 28, 2008
George W. Bush January 23, 2007
George W. Bush January 31, 2006
George W. Bush February 02, 2005
George W. Bush January 20, 2005
George W. Bush January 20, 2004
George W. Bush January 28, 2003
George W. Bush January 29, 2002
George W. Bush January 20, 2001
Bill Clinton January 27, 2000
Bill Clinton January 19, 1999
Bill Clinton January 27, 1998
Bill Clinton February 04, 1997
Bill Clinton January 23, 1996
Bill Clinton January 24, 1995
Bill Clinton January 25, 1994
George H. W. Bush January 28, 1992
George H. W. Bush January 29, 1991
George H. W. Bush January 31, 1990
George H. W. Bush January 20, 198

In [18]:
# Adding political party affiliation to df

party_affil = {
     "Federalist":["George Washington","John Adams"],
    "Democratic-Republican":['Thomas Jefferson',
       'James Madison', 'James Monroe', 'John Quincy Adams'],
    "Democrat":['Andrew Jackson', 'Martin Van Buren',
                'James Knox Polk','Franklin Pierce',
                'James Buchanan','Grover Cleveland',
               'Woodrow Wilson','Franklin D. Roosevelt',
       'Harry S. Truman','John F. Kennedy',
       'Lyndon B. Johnson','Jimmy Carter', 'Bill Clinton', 'Barack Obama'],
    "Republican":['Abraham Lincoln', 'Ulysses S. Grant',
       'Rutherford B. Hayes', 'James A. Garfield','Benjamin Harrison', 'William McKinley', 'Theodore Roosevelt',
       'William Howard Taft','Warren G. Harding', 'Calvin Coolidge', 'Herbert Hoover', 'Dwight D. Eisenhower', 
        'Richard M. Nixon', 'Ronald Reagan', 'George H. W. Bush', 'George W. Bush','Donald Trump', 'Gerald Ford'],
    "Whig":["William Henry Harrison",'Zachary Taylor'],
    }

def get_party_affil(name):
    for party,names in party_affil.items():
        if name in names:
            return party
        
df["Party"] = df["Name"].apply(lambda x : get_party_affil(x))

In [19]:
df

,Name,Date,Speech,Type,Party
0,Donald Trump,"February 05, 2019",\n transcript-innerTranscripttranscript-i...,State of the Union,Republican
1,Donald Trump,"January 30, 2018",\n transcript-innerTranscripttranscript-i...,State of the Union,Republican
2,Donald Trump,"January 20, 2017",\n transcript-innerTranscripttranscript-i...,Inaugural Address,Republican
3,Barack Obama,"January 12, 2016",\n transcript-innerTranscripttranscript-i...,State of the Union,Democrat
4,Barack Obama,"January 20, 2015",\n transcript-innerTranscripttranscript-i...,State of the Union,Democrat
5,Barack Obama,"January 28, 2014",\n transcript-innerTranscripttranscript-i...,State of the Union,Democrat
6,Barack Obama,"February 13, 2013",\n transcript-innerTranscripttranscript-i...,State of the Union,Democrat
7,Barack Obama,"January 21, 2013",\n transcript-innerTranscripttranscript-i...,Inaugural Address,Democrat
8,Barack Obama,"January 24, 2012",\n transcript-innerTranscripttranscript-i...,State of the Union,Democrat
9,Barack Obama,"January 25, 2011",\n transcript-innerTranscripttranscript-i...,State of the Union,Democrat


In [20]:
# Save df to disk

df.to_csv("speeches/data.csv")